**Preparing dataset and essential libraries**

In [106]:
!pip install plotly==4.10.0
!pip install networkx
!pip install "notebook>=5.3" "ipywidgets>=7.2"
!pip install torch torchvision
!pip install stellargraph
!pip install scikit-learn

In [107]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) 

Mounted at /content/drive


In [108]:
import scipy.io
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import tensorflow as tf
import numpy as np
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from gensim.models import Word2Vec

**BlockCatalog**

In [55]:
dataset_path = '/content/drive/My Drive/Anomaly Detection/Code/BlogCatalog.mat'
dataset = scipy.io.loadmat(dataset_path)

G = nx.from_scipy_sparse_matrix(dataset['Network'])
# labels = [x[0] for x in dataset['Label']]
# anomalies = [i for i, x in enumerate(labels) if x == 1]
attributes = dataset['Attributes']
classes = dataset['Class']
wrod2vec_model = Word2Vec.load('/content/drive/My Drive/Anomaly Detection/Code/word2vec_256_blogcatalog.model')

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=uint8)

**Flickr**

In [109]:
dataset_path = '/content/drive/My Drive/Anomaly Detection/Code/Flickr.mat'
dataset = scipy.io.loadmat(dataset_path)

G = nx.from_scipy_sparse_matrix(dataset['Network'])
attributes = dataset['Attributes']
classes = dataset['Label']
wrod2vec_model = Word2Vec.load('/content/drive/My Drive/Anomaly Detection/Code/word2vec_256_flickr.model')

In [110]:
import random
def inject_structural_anomaly(graph, anomaly_nodes, num_cliques, clique_size):
  cliques = [anomaly_nodes[i: i + clique_size] for i in range(0, num_cliques * clique_size, clique_size)]
  for subset in cliques:
    for i in range(len(subset) - 1):
      graph.add_edge(subset[i], subset[i + 1])

def inject_attribute_anomaly(all_nodes, anomaly_nodes, num_cliques, clique_size, k=50):
  global attributes
  random.seed(30)
  for node_i in anomaly_nodes:
    distances = [(idx, np.linalg.norm(attributes[node_i].toarray() - attributes[idx].toarray())) for idx in random.sample(all_nodes, k)]
    distances.sort(key=lambda x: x[1], reverse=True)
    node_j = distances[0][0]
    attributes[node_i] = attributes[node_j]

def extract_anomaly(graph, num_cliques, clique_size):
  global labels
  random.seed(30)
  set_size = num_cliques * clique_size
  selected_nodes = random.sample(graph.nodes, set_size * 2)
  s_anomalies = inject_structural_anomaly(graph, selected_nodes[:set_size], num_cliques, clique_size)
  a_anomalies = inject_attribute_anomaly(graph.nodes, selected_nodes[set_size:], num_cliques, clique_size)
  labels = [0 for i in range(len(graph.nodes))]
  for i in selected_nodes:
    labels[i] = 1
  return selected_nodes

anomalies = extract_anomaly(G, num_cliques=15, clique_size=15)
print(len(anomalies))

/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:118: SparseEfficiencyWarning:

Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.



450


**Graph Embedding**

In [ ]:
from stellargraph import StellarGraph
from stellargraph.data import BiasedRandomWalk

square = StellarGraph.from_networkx(G)
square.info()

rw = BiasedRandomWalk(square)

walks = rw.run(
    nodes=list(G.nodes()),  # root nodes
    length=64,  # maximum length of a random walk
    n=5,  # number of random walks per root node
    p=0.5,  # Defines (unormalised) probability, 1/p, of returning to source node
    q=2.0,  # Defines (unormalised) probability, 1/q, for moving away from source node
)
print("Number of random walks: {}".format(len(walks)))

In [ ]:
GRAPH_EMBEDDING_SIZE = 256
str_walks = [[str(n) for n in walk] for walk in walks]
wrod2vec_model = Word2Vec(str_walks, size=GRAPH_EMBEDDING_SIZE, window=10, min_count=0, sg=1, workers=2, iter=30)
wrod2vec_model.save('/drive/My Drive/Anomaly Detection/Code/word2vec_{}_blogcatalog.model'.format(GRAPH_EMBEDDING_SIZE))

**Setting Hyperparameters**

In [111]:
SEED = 30
EPOCHS = 10
BATCH_SIZE = 128
params = {'batch_size': BATCH_SIZE,
          'shuffle': False,
          'num_workers': 8}
USE_GRAPH = True
USE_ATTRS = True
USE_CLASS = False

**Embedding graph data**

In [112]:
if USE_GRAPH:
  embeddings = wrod2vec_model.wv.vectors
if USE_ATTRS:
  embeddings = np.hstack((attributes.toarray(), wrod2vec_model.wv.vectors)).astype(np.float32)
if USE_CLASS:
  embeddings = np.hstack((classes, embeddings)).astype(np.float32)

# embeddings = attributes.toarray().astype(np.float32)
ATTRIBUTE_SIZE = attributes.shape[1]
GRAPH_SIZE = 256
EMBEDDING_SIZE = embeddings.shape[1]
print(embeddings.shape)
print(ATTRIBUTE_SIZE)

(7575, 12303)
12047


**TENSORFLO‌W VAE**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

def vae_loss_function(recon_x, x, mu, logvar):
    reconstruction_loss = tf.reduce_mean(keras.losses.mse(x, recon_x)) * EMBEDDING_SIZE
    kl_loss = -0.5 * tf.reduce_sum(1 + logvar - tf.square(mu) - tf.exp(logvar))
    return reconstruction_loss, kl_loss

In [ ]:
from keras.utils import plot_model
import numpy as np
import scipy.stats
from sklearn.metrics import roc_auc_score

total_hidden_layers = [[1]]
latent_dims = [1]

for i in range(len(latent_dims)):
  hidden_layers = total_hidden_layers[i]
  latent_dim = latent_dims[i]

  encoder_inputs = tf.keras.Input(shape=(EMBEDDING_SIZE,), name="encoder_input")
  h1 = layers.Dense(hidden_layers[0], activation='relu')(encoder_inputs)
  for i in range(1, len(hidden_layers)):
      h1 = layers.Dense(hidden_layers[i], activation='relu')(h1)
  z_mean = layers.Dense(latent_dim, name="z_mean")(h1)
  z_log_var = layers.Dense(latent_dim, name="z_log_var")(h1)
  z = Sampling()([z_mean, z_log_var])
  encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
  encoder.summary()

  latent_inputs = tf.keras.Input(shape=(latent_dim,), name="z_sampling")
  h2 = layers.Dense(hidden_layers[-1], activation="relu")(latent_inputs)
  for i in range(len(hidden_layers) - 2, -1, -1):
    h2 = layers.Dense(hidden_layers[i], activation='relu')(h2)
  decoder_outputs = layers.Dense(EMBEDDING_SIZE, activation='sigmoid')(h2)
  decoder = tf.keras.Model(inputs=latent_inputs, outputs=decoder_outputs, name="decoder")
  decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
  decoder.summary()
    
  #VAE model.
  class VAE(keras.Model):
      def __init__(self, encoder, decoder, **kwargs):
          super(VAE, self).__init__(**kwargs)
          self.encoder = encoder
          self.decoder = decoder

      def train_step(self, data):
          with tf.GradientTape() as tape:
              z_mean, z_log_var, z = encoder(data)
              reconstruction = decoder(z)
              reconstruction_loss, kl_loss = vae_loss_function(reconstruction, data, z_mean, z_log_var)
              total_loss = reconstruction_loss + kl_loss
          grads = tape.gradient(total_loss, self.trainable_weights)
          self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
          return {
              "loss": total_loss,
              "reconstruction_loss": reconstruction_loss,
              "kl_loss": kl_loss,
          }

  vae = VAE(encoder, decoder)
  vae.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3))
  vae.fit(embeddings, epochs=EPOCHS, batch_size=BATCH_SIZE)
  
  losses = []
  latents = []

  for data, label in zip(embeddings, labels):
    data = np.array([data])
    z_mean, z_log_var, z = encoder(data)
    reconstruction = decoder(z)
    reconstruction_loss, kl_loss = vae_loss_function(reconstruction, data, z_mean, z_log_var)
    total_loss = reconstruction_loss + kl_loss
    losses.append(total_loss.numpy())

  np_losses = np.array(losses)
  np_losses = np_losses / np.max(np_losses)
  auc = roc_auc_score(np.array(labels), np_losses)
  def check_intersection(a):
    c = 0
    for idx in a:
      if labels[idx] == 1:
        c += 1
    return c
  top_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:len(anomalies)]
  print('Latent Space: {}'.format(latent_dim))
  print('Top {}: '.format(len(anomalies)), check_intersection(top_indices))
  print('AUC:‌ ', auc)
  print('=' * 50)
  top50_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:50]
  top100_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:100]
  top200_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:200]
  top300_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:300]
  print('Culmulative Percision')
  print('for T=50: {}'.format(check_intersection(top50_indices) / 50))
  print('for T=100: {}'.format(check_intersection(top100_indices) / 100))
  print('for T=200: {}'.format(check_intersection(top200_indices) / 200))
  print('for T=300: {}'.format(check_intersection(top300_indices) / 300))
  print('=' * 50)
  print('Culmulative Recall')
  print('for T=50: {}'.format(check_intersection(top50_indices) / len(anomalies)))
  print('for T=100: {}'.format(check_intersection(top100_indices) / len(anomalies)))
  print('for T=200: {}'.format(check_intersection(top200_indices) / len(anomalies)))
  print('for T=300: {}'.format(check_intersection(top300_indices) / len(anomalies)))
  print('=' * 50 + '\n')


Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 12303)]      0                                            
__________________________________________________________________________________________________
dense_92 (Dense)                (None, 1)            12304       encoder_input[0][0]              
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 1)            2           dense_92[0][0]                   
__________________________________________________________________________________________________
z_log_var (Dense)               (None, 1)            2           dense_92[0][0]                   
____________________________________________________________________________________________

In [ ]:
import numpy as np
import scipy.stats
losses = []
latents = []

for data, label in zip(embeddings, labels):
  data = np.array([data])
  z_mean, z_log_var, z = encoder(data)
  reconstruction = decoder(z)
  reconstruction_loss, kl_loss = vae_loss_function(reconstruction, data, z_mean, z_log_var)
  total_loss = reconstruction_loss + kl_loss
  losses.append(total_loss.numpy())
print(losses)

[5864.1987, 11.435816, 36.367367, 32.03388, 51.16141, 39.348648, 15.51924, 43.360046, 23.931778, 44.70403, 3.1951704, 115.68372, 71.009384, 16.251299, 127.66486, 323.31393, 11.255731, 198.77586, 74.006424, 11.44533, 29.29516, 55.852505, 24.032188, 11.294098, 16.36106, 44.160084, 38.370705, 12.133015, 13.274379, 37.2703, 22.324028, 29.954224, 72.61573, 90.82815, 199.93419, 84.775215, 65.084915, 27.330288, 137.47144, 30.558144, 8902.978, 230.7187, 35.000942, 363.01794, 90.13487, 35.296234, 110.70323, 13.447481, 25.051922, 23.059738, 34.630596, 22.634619, 66.268654, 14.276156, 139.7091, 32.90358, 15.372695, 105.877304, 85.174706, 29.245241, 23.42688, 34.326317, 3.2380002, 46.21617, 19.379086, 29.096037, 26.31483, 61.158062, 90.06765, 45.791714, 58.90593, 384.91736, 16.18271, 41.28853, 38.508278, 36.363045, 49.042995, 363.87, 140.5821, 34.320847, 46.162025, 17.485723, 9.440512, 68.89494, 30.119127, 78.168465, 28.31949, 45.01703, 58.28995, 1536.7601, 365.27582, 7.254494, 32.343098, 64.01092

**Calculating Metrics**

In [ ]:
from sklearn.metrics import roc_auc_score
np_losses = np.array(losses)
np_losses = np_losses / np.max(np_losses)
auc = roc_auc_score(np.array(labels), np_losses)
def check_intersection(a):
  c = 0
  for idx in a:
    if labels[idx] == 1:
      c += 1
  return c
top_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:len(anomalies)]
print('Top {}: '.format(len(anomalies)), check_intersection(top_indices))
print('AUC:‌ ', auc)
print('=' * 50)
top50_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:50]
top100_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:100]
# top150_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:150]
top200_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:200]
top300_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:300]
print('Culmulative Percision')
print('for T=50: {}'.format(check_intersection(top50_indices) / 50))
print('for T=100: {}'.format(check_intersection(top100_indices) / 100))
# print('for T=150: {}'.format(check_intersection(top150_indices) / 150))
print('for T=200: {}'.format(check_intersection(top200_indices) / 200))
print('for T=300: {}'.format(check_intersection(top300_indices) / 300))
print('=' * 50)
print('Culmulative Recall')
print('for T=50: {}'.format(check_intersection(top50_indices) / len(anomalies)))
print('for T=100: {}'.format(check_intersection(top100_indices) / len(anomalies)))
# print('for T=150: {}'.format(check_intersection(top150_indices) / len(anomalies)))
print('for T=200: {}'.format(check_intersection(top200_indices) / len(anomalies)))
print('for T=300: {}'.format(check_intersection(top300_indices) / len(anomalies)))
print('=' * 50)


Top 450:  211
AUC:‌  0.7463666276803118
Culmulative Percision
for T=50: 0.74
for T=100: 0.77
for T=200: 0.66
for T=300: 0.5833333333333334
Culmulative Recall
for T=50: 0.08222222222222222
for T=100: 0.1711111111111111
for T=200: 0.29333333333333333
for T=300: 0.3888888888888889


**Test Isolation Forest**

In [ ]:
from sklearn.ensemble import IsolationForest
latents = np.squeeze(np.array(latents))
# clf = IsolationForest(random_state=0).fit(latents[:math.ceil(0.9 * node_numbers)])
clf = IsolationForest(random_state=0).fit(latents)
preds = -clf.score_samples(latents)
from sklearn.metrics import roc_auc_score
roc_auc_score(np.array(labels), preds)

In [ ]:
import random
x_normal_sampled = random.sample(x_normal_population, 300)
plt.hist(x_normal_sampled, bins=10, alpha=0.5,
         histtype='stepfilled', color='steelblue',
         edgecolor='none');
plt.hist(x_anomaly_population, bins=10, alpha=0.5,
         histtype='stepfilled', color='red',
         edgecolor='none');
print('Normal Node')
print('mean:‌ ', mean_normal)
print('variance:‌ ', std_normal)
print('-' * 40)
print('Anomaly Node')
print('mean:‌ ', mean_anomaly)
print('variance:‌ ', std_anomaly)
plt.show()

In [ ]:
y_normal = scipy.stats.norm.pdf(x_normal_sampled,mean_normal,std_normal)
y_anomaly = scipy.stats.norm.pdf(x_anomaly_population,mean_anomaly,std_anomaly)
kwargs = dict(histtype='stepfilled', alpha=0.3, bins=40)
plt.scatter(x_normal_sampled,y_normal, color='blue', )
plt.scatter(x_anomaly_population,y_anomaly, color='red')
plt.grid()
plt.show()

**Building the architecture of models (PYTORCH)**

In [43]:
import pandas as pd
import tensorflow as tf
from torchvision import datasets, transforms
import math
from torch import nn

np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
  torch.backends.cudnn.deterministic = True
  torch.cuda.manual_seed(SEED)
# device = torch.device("cuda" if not torch.cuda.is_available() else "cpu")
device = 'cpu'

print('Run on {}'.format(device))

class GraphDataset(torch.utils.data.Dataset):
  def __init__(self, vecs, labels):
        self.vectors = vecs
        self.labels = labels

  def __len__(self):
        return len(self.vectors)

  def __getitem__(self, index):
        return self.vectors[index], self.labels[index]

training_set = GraphDataset(embeddings, labels)
training_generator = torch.utils.data.DataLoader(training_set, **params)
validating_set = GraphDataset(embeddings[:1000], labels[:1000])
validating_generator = torch.utils.data.DataLoader(validating_set, **params)

hidden_layers = [32]
class VAE(nn.Module):
    def __init__(self, embedding_size, latent_size):
        super(VAE, self).__init__()
        self.embedding_size = embedding_size
        self.fc_encode_list = []
        self.fc_decode_list = []
        self.fc0 = nn.Linear(embedding_size, hidden_layers[0])
        self.dropout = nn.Dropout(p=0.5)
        for i in range(len(hidden_layers) - 1):
            self.fc_encode_list.append(nn.Linear(hidden_layers[i], hidden_layers[i + 1]))
        self.fcl1 = nn.Linear(hidden_layers[-1],  latent_size)
        self.fcl2 = nn.Linear(hidden_layers[-1], latent_size)
        self.fc3 = nn.Linear(latent_size, hidden_layers[-1])
        for i in range(len(hidden_layers) - 1, 0, -1):
            self.fc_decode_list.append(nn.Linear(hidden_layers[i], hidden_layers[i - 1]))
        self.fc4 = nn.Linear(hidden_layers[0], embedding_size)

    def encode(self, x):
        h = F.relu(self.fc0(x))
        # h = self.dropout(h)
        for fc in self.fc_encode_list:
          h = F.relu(fc(h))
          # h = self.dropout(h)
        return self.fcl1(h), self.fcl2(h)
    
    def getZ(self, x):
        mu, logvar = self.encode(x)
        return self.reparameterize(mu, logvar)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc3(z))
        # h = self.dropout(h)
        for fc in self.fc_decode_list:
          h = F.relu(fc(h))
          # h = self.dropout(h)
        return torch.sigmoid(self.fc4(h))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, self.embedding_size))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

def loss_function(recon_x, x, mu, logvar):
    BCE = F.mse_loss(recon_x, x.view(-1, EMBEDDING_SIZE), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD


model = VAE(embedding_size=EMBEDDING_SIZE, latent_size=1).to(device)

optimizer = optim.Adam(model.parameters(), lr=2e-3)

def do_train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, label) in enumerate(training_generator):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 4 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(training_generator.dataset),
                100. * batch_idx / len(training_generator),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(training_generator.dataset)))

def do_test():
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, label) in enumerate(validating_generator):
            data = data.to(device) 
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
    test_loss /= len(validating_generator.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


import time
for epoch in range(1, 5):
    t1 = time.time()
    do_train(epoch)
    do_test()
    t2 = time.time()
    print('Time for this epoch: {}'.format(t2 - t1))

Run on cpu
Train Epoch: 1 [0/7575 (0%)]	Loss: 3382.447998
Train Epoch: 1 [256/7575 (3%)]	Loss: 2957.288086
Train Epoch: 1 [512/7575 (7%)]	Loss: 2784.129150
Train Epoch: 1 [768/7575 (10%)]	Loss: 2515.630127
Train Epoch: 1 [1024/7575 (13%)]	Loss: 2283.382080
Train Epoch: 1 [1280/7575 (17%)]	Loss: 2012.072998
Train Epoch: 1 [1536/7575 (20%)]	Loss: 1794.944458
Train Epoch: 1 [1792/7575 (24%)]	Loss: 1434.620117
Train Epoch: 1 [2048/7575 (27%)]	Loss: 1101.358276
Train Epoch: 1 [2304/7575 (30%)]	Loss: 952.446228
Train Epoch: 1 [2560/7575 (34%)]	Loss: 745.536011
Train Epoch: 1 [2816/7575 (37%)]	Loss: 707.278870
Train Epoch: 1 [3072/7575 (40%)]	Loss: 482.726593
Train Epoch: 1 [3328/7575 (44%)]	Loss: 359.046600
Train Epoch: 1 [3584/7575 (47%)]	Loss: 478.544556
Train Epoch: 1 [3840/7575 (50%)]	Loss: 450.642334
Train Epoch: 1 [4096/7575 (54%)]	Loss: 369.760651
Train Epoch: 1 [4352/7575 (57%)]	Loss: 413.263245
Train Epoch: 1 [4608/7575 (61%)]	Loss: 319.280823
Train Epoch: 1 [4864/7575 (64%)]	Loss: 

**Parsa Version**

In [113]:
import pandas as pd
import tensorflow as tf
from torchvision import datasets, transforms
import math
from torch import Tensor
from torch.autograd import Variable

np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
  torch.backends.cudnn.deterministic = True
  torch.cuda.manual_seed(SEED)
# device = torch.device("cuda" if not torch.cuda.is_available() else "cpu")
device = 'cpu'

print('Run on {}'.format(device))

class GraphDataset(torch.utils.data.Dataset):
  def __init__(self, vecs, labels):
        self.vectors = vecs
        self.labels = labels

  def __len__(self):
        return len(self.vectors)

  def __getitem__(self, index):
        return self.vectors[index], self.labels[index]

training_set = GraphDataset(embeddings, labels)
training_generator = torch.utils.data.DataLoader(training_set, **params)
validating_set = GraphDataset(embeddings[:1000], labels[:1000])
validating_generator = torch.utils.data.DataLoader(validating_set, **params)

hidden_layers = [128]
attribute_hidden_layers = [64]
graph_hidden_layers = [64]

class VAE(nn.Module):
    def __init__(self, embedding_size, attribute_size, latent_size):
        super(VAE, self).__init__()
        self.embedding_size = embedding_size
        self.attribute_size = attribute_size
        self.graph_size = embedding_size - attribute_size
        self.dropout = nn.Dropout(p=0.3)
        self.att_fc_encode_list = []
        self.graph_fc_encode_list = []
        self.att_fc_decode_list = []
        self.graph_fc_decode_list = []
      
        self.att_fc0 = nn.Linear(attribute_size, attribute_hidden_layers[0])
        for i in range(len(attribute_hidden_layers) - 1):
            self.att_fc_encode_list.append(nn.Linear(attribute_hidden_layers[i], attribute_hidden_layers[i + 1]))

        self.graph_fc0 = nn.Linear(self.graph_size, graph_hidden_layers[0])
        for i in range(len(graph_hidden_layers) - 1):
            self.graph_fc_encode_list.append(nn.Linear(graph_hidden_layers[i], graph_hidden_layers[i + 1]))

        self.fcl1 = nn.Linear(graph_hidden_layers[-1] + attribute_hidden_layers[-1], latent_size)
        self.fcl2 = nn.Linear(graph_hidden_layers[-1] + attribute_hidden_layers[-1], latent_size)
        
        self.att_fc3 = nn.Linear(latent_size, attribute_hidden_layers[-1])
        self.graph_fc3 = nn.Linear(latent_size, graph_hidden_layers[-1])

        for i in range(len(attribute_hidden_layers) - 1, 0, -1):
            self.att_fc_decode_list.append(nn.Linear(attribute_hidden_layers[i], attribute_hidden_layers[i - 1]))
        
        for i in range(len(graph_hidden_layers) - 1, 0, -1):
            self.graph_fc_decode_list.append(nn.Linear(graph_hidden_layers[i], graph_hidden_layers[i - 1]))

        self.att_fc4 = nn.Linear(attribute_hidden_layers[0], attribute_size)
        self.graph_fc4 = nn.Linear(graph_hidden_layers[0], self.graph_size)


    def encode(self, x):
        encoded_att = F.relu(self.att_fc0(x[:, :self.attribute_size]))
        encoded_att = self.dropout(encoded_att)
        for fc in self.att_fc_encode_list:
          encoded_att = F.relu(fc(encoded_att))
          encoded_att = self.dropout(encoded_att)
      
        encoded_graph = F.relu(self.graph_fc0(x[:, self.attribute_size:]))
        encoded_graph = self.dropout(encoded_graph)
        for fc in self.graph_fc_encode_list:
          encoded_graph = F.relu(fc(encoded_graph))
          encoded_graph = self.dropout(encoded_graph)

        encoded_emb = torch.cat((encoded_att, encoded_graph), 1)
        # h = F.relu(self.fc0())

        return self.dropout(self.fcl1(encoded_emb)), self.dropout(self.fcl2(encoded_emb))
    
    def getZ(self, x):
        mu, logvar = self.encode(x)
        return self.reparameterize(mu, logvar)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        decoded_att = F.relu(self.att_fc3(z))
        decoded_att = self.dropout(decoded_att)

        for fc in self.att_fc_decode_list:
          decoded_att = F.relu(fc(decoded_att))
          decoded_att = self.dropout(decoded_att)

        decoded_att = torch.sigmoid(self.att_fc4(decoded_att))
        decoded_att = self.dropout(decoded_att)

        decoded_graph = F.relu(self.graph_fc3(z))
        decoded_graph = self.dropout(decoded_graph)

        for fc in self.graph_fc_decode_list:
          decoded_graph = F.relu(fc(decoded_graph))
          decoded_graph = self.dropout(decoded_graph)

        decoded_graph = torch.sigmoid(self.graph_fc4(decoded_graph))
        decoded_graph = self.dropout(decoded_graph)

        return decoded_att, decoded_graph
        # return torch.cat((decoded_att, decoded_graph), 1)

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, self.embedding_size))
        z = self.reparameterize(mu, logvar)
        decoded_att, decoded_graph = self.decode(z)
        # return self.decode(z), mu, logvar
        return decoded_att, decoded_graph, mu, logvar


def loss_function(att_recon_x, graph_recon_x, x, mu, logvar):
    att_BCE = F.mse_loss(att_recon_x, x[:, :ATTRIBUTE_SIZE].view(-1, ATTRIBUTE_SIZE), reduction='sum')
    graph_BCE = F.mse_loss(graph_recon_x, x[:, ATTRIBUTE_SIZE:].view(-1, EMBEDDING_SIZE - ATTRIBUTE_SIZE), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return 5 * att_BCE + graph_BCE + KLD


model = VAE(embedding_size=EMBEDDING_SIZE, attribute_size=ATTRIBUTE_SIZE, latent_size=64).to(device)

optimizer = optim.Adam(model.parameters(), lr=2e-3)

def do_train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, label) in enumerate(training_generator):
        data = data.to(device)
        # print(data)
        optimizer.zero_grad()
        att_recon_batch, graph_recon_batch, mu, logvar = model(data)
        loss = loss_function(att_recon_batch, graph_recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 4 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(training_generator.dataset),
                100. * batch_idx / len(training_generator),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(training_generator.dataset)))

def do_test():
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, label) in enumerate(validating_generator):
            data = data.to(device) 
            # recon_batch, mu, logvar = model(data)
            att_recon_batch, graph_recon_batch, mu, logvar = model(data)
            test_loss += loss_function(att_recon_batch, graph_recon_batch, data, mu, logvar).item()
    test_loss /= len(validating_generator.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


import time
for epoch in range(1, 3):
    t1 = time.time()
    do_train(epoch)
    do_test()
    t2 = time.time()
    print('Time for this epoch: {}'.format(t2 - t1))

Run on cpu
Train Epoch: 1 [0/7575 (0%)]	Loss: 22802.083984
Train Epoch: 1 [512/7575 (7%)]	Loss: 20228.564453
Train Epoch: 1 [1024/7575 (13%)]	Loss: 17191.457031
Train Epoch: 1 [1536/7575 (20%)]	Loss: 13379.463867
Train Epoch: 1 [2048/7575 (27%)]	Loss: 8114.072266
Train Epoch: 1 [2560/7575 (33%)]	Loss: 16879.990234
Train Epoch: 1 [3072/7575 (40%)]	Loss: 7390.316895
Train Epoch: 1 [3584/7575 (47%)]	Loss: 4197.014160
Train Epoch: 1 [4096/7575 (53%)]	Loss: 1298.916260
Train Epoch: 1 [4608/7575 (60%)]	Loss: 1551.957764
Train Epoch: 1 [5120/7575 (67%)]	Loss: 831.672852
Train Epoch: 1 [5632/7575 (73%)]	Loss: 1620.640137
Train Epoch: 1 [6144/7575 (80%)]	Loss: 1030.984253
Train Epoch: 1 [6656/7575 (87%)]	Loss: 446.004150
Train Epoch: 1 [7168/7575 (93%)]	Loss: 777.622620
====> Epoch: 1 Average loss: 44005.0748
====> Test set loss: 968.1224
Time for this epoch: 5.471452236175537
Train Epoch: 2 [0/7575 (0%)]	Loss: 1037.838745
Train Epoch: 2 [512/7575 (7%)]	Loss: 1144.750854
Train Epoch: 2 [1024/75

In [103]:
from torch.autograd import Variable
from torch import Tensor
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
losses = []
latents = []
tmp = None
for data, label in zip(embeddings, labels):
  data = Variable(Tensor([data]))
  # print('data size,', data[0][0:300].shape)
  # if tmp != None:
  #   catt = torch.cat((data, tmp), 1)
  #   print(data.shape)
  # tmp = data
  att_recon_batch, graph_recon_batch, mu, logvar = model(data)
  test_loss = loss_function(att_recon_batch, graph_recon_batch, data, mu, logvar).item()
  losses.append(test_loss)
  latents.append(model.getZ(data).detach().numpy())

In [104]:
from sklearn.metrics import roc_auc_score
np_losses = np.array(losses)
np_losses = np_losses / np.max(np_losses)
auc = roc_auc_score(np.array(labels), np_losses)
# auc = roc_auc_score(np.array(dataset['Label']), np_losses)
def check_intersection(a):
  c = 0
  for idx in a:
    if labels[idx] == 1:
      c += 1
  return c
top_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:len(anomalies)]
print('Top {}: '.format(len(anomalies)), check_intersection(top_indices))
print('AUC:‌ ', auc)
print('=' * 50)
top50_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:50]
top100_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:100]
# top150_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:150]
top200_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:200]
top300_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:300]
print('Culmulative Percision')
print('for T=50: {}'.format(check_intersection(top50_indices) / 50))
print('for T=100: {}'.format(check_intersection(top100_indices) / 100))
# print('for T=150: {}'.format(check_intersection(top150_indices) / 150))
print('for T=200: {}'.format(check_intersection(top200_indices) / 200))
print('for T=300: {}'.format(check_intersection(top300_indices) / 300))
print('=' * 50)
print('Culmulative Recall')
print('for T=50: {}'.format(check_intersection(top50_indices) / len(anomalies)))
print('for T=100: {}'.format(check_intersection(top100_indices) / len(anomalies)))
# print('for T=150: {}'.format(check_intersection(top150_indices) / len(anomalies)))
print('for T=200: {}'.format(check_intersection(top200_indices) / len(anomalies)))
print('for T=300: {}'.format(check_intersection(top300_indices) / len(anomalies)))
print('=' * 50)


Top 300:  134
AUC:‌  0.7436376633986929
Culmulative Percision
for T=50: 0.62
for T=100: 0.63
for T=200: 0.535
for T=300: 0.44666666666666666
Culmulative Recall
for T=50: 0.10333333333333333
for T=100: 0.21
for T=200: 0.3566666666666667
for T=300: 0.44666666666666666


In [92]:
torch.save(model, '/content/drive/My Drive/Anomaly Detection/Code/models/blog_catalog_H32_L1_E1')

In [ ]:
model2 = torch.load('/content/drive/My Drive/Anomaly Detection/Code/models/blog_catalog')

In [114]:
from torch.autograd import Variable
from torch import Tensor
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats

iterations = 15
tops = []
aucs = []
p50 = []
p100 = []
p200 = []
p300 = []
r50 = []
r100 = []
r200 = []
r300 = []

for i in range(iterations):
  losses = []
  latents = []
  for data, label in zip(embeddings, labels):
    data = Variable(Tensor([data]))
    # recon_batch, mu, logvar = model(data)
    # test_loss = loss_function(recon_batch, data, mu, logvar).item()
    att_recon_batch, graph_recon_batch, mu, logvar = model(data)
    test_loss = loss_function(att_recon_batch, graph_recon_batch, data, mu, logvar).item()
    losses.append(test_loss)

  np_losses = np.array(losses)
  np_losses = np_losses / np.max(np_losses)
  auc = roc_auc_score(np.array(labels), np_losses)
  # auc = roc_auc_score(np.array(dataset['Label']), np_losses)
  def check_intersection(a):
    c = 0
    for idx in a:
      if labels[idx] == 1:
        c += 1
    return c
  top_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:len(anomalies)]

  tops.append(check_intersection(top_indices))
  aucs.append(auc)
  top50_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:50]
  top100_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:100]
  # top150_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:150]
  top200_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:200]
  top300_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:300]

  p50.append(check_intersection(top50_indices) / 50)
  p100.append(check_intersection(top100_indices) / 100)
  p200.append(check_intersection(top200_indices) / 200)
  p300.append(check_intersection(top300_indices) / 300)
  r50.append(check_intersection(top50_indices) / len(anomalies))
  r100.append(check_intersection(top100_indices) / len(anomalies))
  r200.append(check_intersection(top200_indices) / len(anomalies))
  r300.append(check_intersection(top300_indices) / len(anomalies))


In [ ]:
aucs

[0.7309027777777778,
 0.7381678921568628,
 0.7439385893246186,
 0.7390495642701526,
 0.7303043300653596,
 0.7278513071895425,
 0.7413017429193899,
 0.73,
 0.7427975217864924,
 0.7356015114379084,
 0.7358775871459695,
 0.7376477396514161,
 0.7383871187363834,
 0.727542211328976,
 0.7352457788671024]

In [115]:
print('Max AUC: ', np.max(aucs))
# print('Min AUC: ', np.min(aucs))
# print('Mean AUC: ', np.mean(aucs))

print('=' * 50)

print('Culmulative Percision')
print('for T=50: {}'.format(np.max(p50)))
print('for T=100: {}'.format(np.max(p100)))
# print('for T=150: {}'.format(check_intersection(top150_indices) / 150))
print('for T=200: {}'.format(np.max(p200)))
print('for T=300: {}'.format(np.max(p300)))
print('=' * 50)
print('Culmulative Recall')
print('for T=50: {}'.format(np.max(r50)))
print('for T=100: {}'.format(np.max(r100)))
# print('for T=150: {}'.format(check_intersection(top150_indices) / len(anomalies)))
print('for T=200: {}'.format(np.max(r200)))
print('for T=300: {}'.format(np.max(r300)))



Max AUC:  0.7480935672514619
Culmulative Percision
for T=50: 0.74
for T=100: 0.77
for T=200: 0.66
for T=300: 0.5833333333333334
Culmulative Recall
for T=50: 0.08222222222222222
for T=100: 0.1711111111111111
for T=200: 0.29333333333333333
for T=300: 0.3888888888888889


In [ ]:
print('Top {}: '.format(len(anomalies)), check_intersection(top_indices))
print('AUC:‌ ', auc)
print('=' * 50)
top50_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:50]
top100_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:100]
# top150_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:150]
top200_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:200]
top300_indices = sorted(range(len(losses)), key=lambda i: losses[i], reverse=True)[:300]
print('Culmulative Percision')
print('for T=50: {}'.format(check_intersection(top50_indices) / 50))
print('for T=100: {}'.format(check_intersection(top100_indices) / 100))
# print('for T=150: {}'.format(check_intersection(top150_indices) / 150))
print('for T=200: {}'.format(check_intersection(top200_indices) / 200))
print('for T=300: {}'.format(check_intersection(top300_indices) / 300))
print('=' * 50)
print('Culmulative Recall')
print('for T=50: {}'.format(check_intersection(top50_indices) / len(anomalies)))
print('for T=100: {}'.format(check_intersection(top100_indices) / len(anomalies)))
# print('for T=150: {}'.format(check_intersection(top150_indices) / len(anomalies)))
print('for T=200: {}'.format(check_intersection(top200_indices) / len(anomalies)))
print('for T=300: {}'.format(check_intersection(top300_indices) / len(anomalies)))
print('=' * 50)

**Visualizing the graph and more analysis.**



In [ ]:
# pos = nx.spring_layout(G, scale=0.2)
# edge_x = []
# edge_y = []
# for edge in G.edges():
#     x0, y0 = tuple(pos[edge[0]])
#     x1, y1 = tuple(pos[edge[1]])
#     edge_x.append(x0)
#     edge_x.append(x1)
#     edge_x.append(None)
#     edge_y.append(y0)
#     edge_y.append(y1)
#     edge_y.append(None)

# edge_trace = go.Scatter(
#     x=edge_x, y=edge_y,
#     line=dict(width=0.5, color='#888'),
#     hoverinfo='none',
#     mode='lines')

# node_x = []
# node_y = []
# for node in G.nodes():
#     x, y = pos[node]
#     node_x.append(x)
#     node_y.append(y)

# node_trace = go.Scatter(
#     x=node_x, y=node_y,
#     mode='markers',
#     hoverinfo='text',
#     marker=dict(
#         showscale=True,
#         colorscale='Viridis',
#         reversescale=True,
#         color=[],
#         size=10,
#         colorbar=dict(
#             thickness=15,
#             title='Node Connections',
#             xanchor='left',
#             titleside='right'
#         ),
#         line_width=2))

# node_adjacencies = []
# node_text = []
# for node, adjacencies in enumerate(G.adjacency()):
#     node_adjacencies.append(len(adjacencies[1]))
#     node_text.append('# of connections: '+str(len(adjacencies[1])))

# node_trace.marker.color = node_adjacencies
# node_trace.text = node_text
# fig = go.Figure(data=[edge_trace, node_trace],
#              layout=go.Layout(
#                 title='<br>Network graph made with Python',
#                 titlefont_size=16,
#                 showlegend=False,
#                 hovermode='closest',
#                 margin=dict(b=20,l=5,r=5,t=40),
#                 xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
#                 yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
#                 )
# fig.show()